In [1]:
#импорт библиотек, загрузка модели, весов, разметки клаассов
import torch
import numpy as np
import pandas as pd
device = torch.device("cpu")
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AdamW, get_scheduler
model_name = "sberbank-ai/sbert_large_nlu_ru"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=660).to(device)
from sklearn import preprocessing
Label_encoder = preprocessing.LabelEncoder()
Label_encoder.classes_ = np.load('./cl_classes2306.npy', allow_pickle=True)
model.load_state_dict(torch.load("../../best_model2306/pytorch_model.bin", map_location=device))

C:\Users\yeril\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sberbank-ai/sbert_large_nlu_ru and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [2]:
#импорт описания классов (выдрал с альта софта)
dscr = pd.read_csv("../data/desc.csv", sep=';', names=['id', 'label'], dtype={'id': str, 'label': str})

In [3]:
#ищем классы нейросети, к которым нет описания.
withlabels = list(dscr['id'])
justids = list(Label_encoder.classes_)
for each in justids:
    if each not in withlabels:
        print(each)

0391
0481
0830
1012
1018
1062
1068
5022
5032
5035
5052
5058
5075
6908


In [3]:
#функция предсказания одного класса (аргмакс). По умолчанию с кодом класса дает описание класса
def predict_class(text, desc=True):
    model.to(torch.device('cpu'))
    inputs = tokenizer(text, truncation = True, max_length=100, padding='max_length', return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
        predicted_class_id = logits.argmax().item()
        if desc==True:
            result = dict()
            id = Label_encoder.inverse_transform([predicted_class_id])[0]
            result[id] = dscr[dscr['id']==id].iloc[0]['label']
            return result
        else:
            return Label_encoder.inverse_transform([predicted_class_id])[0]

In [21]:
#Пример с описанием
predict_class('КОНДИТЕРСКИЕ ИЗДЕЛИЯ В ВИДЕ РЕЗИНКИ И ЖЕЛЕ, НЕ СОДЕРЖАЩИЕ КАКАО')

{'1704': 'Кондитерские изделия из сахара (включая белый шоколад), не содержащие какао'}

In [20]:
#Пример без описания
predict_class('КОНДИТЕРСКИЕ ИЗДЕЛИЯ В ВИДЕ РЕЗИНКИ И ЖЕЛЕ, НЕ СОДЕРЖАЩИЕ КАКАО', desc=False)

'1704'

In [4]:
#функция предикта классов с вероятностью (софтмакс), по умолчанию 5 классов
def predict_prob(text, qtty=5):
    model.to(torch.device('cpu'))
    inputs = tokenizer(text, truncation = True, max_length=100, padding='max_length', return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    result = dict()
    p = torch.nn.functional.softmax(logits, dim=1)
    for i in range(qtty):
        a = p.argmax().item()
        result[Label_encoder.inverse_transform([a])[0]] = p[0][a].item()
        p[0][a] = 0
    return result

In [22]:
#Пример с 3 классами
print(predict_prob('трубы сантехнические пластиковые ', 3))

{'3917': 0.7268840670585632, '7304': 0.01337417121976614, '4009': 0.010102425701916218}


In [15]:
#Пример с 5 классами
print(predict_prob('трубы металлические ', 5))

{'7309': 0.9046436548233032, '3919': 0.010384665802121162, '7310': 0.00989900715649128, '8311': 0.003736451966688037, '7311': 0.0020984946750104427}


In [5]:
#функция предикта классов и описания с вероятностью, по умолчанию 5 шт. Использует функцию predict_prob.
#возвращает список списков
def predict_prob_with_descr(text, qtty=5):
    probs = predict_prob(text, qtty=qtty)
    #result = np.array()
    result = list()
    for each in probs:
        result.append([each, dscr[dscr['id']==each].iloc[0]['label'], probs[each]])
    return result

In [23]:
#Пример с 5 классами.
predict_prob_with_descr('покрышки')

[['4011',
  'Шины и покрышки пневматические резиновые новые',
  0.7416127324104309],
 ['4016',
  'Изделия из вулканизованной резины, кроме твердой резины, прочие',
  0.02087979018688202],
 ['8708',
  'Части и принадлежности моторных транспортных средств товарных позиций 8701 - 8705',
  0.011835361830890179],
 ['9401',
  'Мебель для сидения (кроме указанной в товарной позиции 9402), трансформируемая или не трансформируемая в кровати, и ее части',
  0.011823809705674648],
 ['7312',
  'Скрученная проволока, тросы, канаты, плетеные шнуры, стропы и аналогичные изделия, из черных металлов, без электрической изоляции',
  0.009819085709750652]]

In [45]:
#причесанный в пандас пример
pd.DataFrame(predict_prob_with_descr('орехи'), columns=['ID', 'LABEL', 'PROB'])

,ID,LABEL,PROB
0,1202,"Арахис, нежареный или не приготовленный каким-...",0.024651
1,0808,"Яблоки, груши и айва, свежие",0.014857
2,2001,"Овощи, фрукты, орехи и другие съедобные части ...",0.012068
3,8311,"Проволока, прутки, трубы, пластины, электроды ...",0.010070
4,8506,Первичные элементы и первичные батареи,0.009615


In [6]:
def consolelog(probs):
    print('prob-ty', 'Code', 'Description', sep=':    ')
    #print('____________________________________________________________________')
    for each in probs:
        if len(each[1]) < 40:
            des = each[1]
        else:
            des = each[1][:40] + '...'
        print(f"{each[2]:.{5}f}", each[0], des, sep=':    ')
        #print('____________________________________________________________________')

In [16]:
consolelog(predict_prob_with_descr("гандон", 5))

prob-ty:    Code:    Description
0.09167:    4016:    Изделия из вулканизованной резины, кроме...
0.05223:    9603:    Метлы, щетки (включая щетки, являющиеся ...
0.03173:    7320:    Пружины, рессоры и листы для них, из чер...
0.02111:    9404:    Основы матрацные; принадлежности постель...
0.02036:    9503:    Трехколесные велосипеды, самокаты, педал...


In [8]:
#загрузим обучающий датасет
df = pd.read_csv("../data/mergedcleared0407.csv", sep=';', names=['id', 'label'], dtype={'id': str, 'label': str})
#df = pd.read_csv("../data/firstset.csv", sep=';', names=['id', 'label'], dtype={'id': str, 'label': str})
#функция для вывода обучающих данных по коду. При pandas=False возвращает numpy массив
def whats_data(id, pandas=True):
    if pandas == True:
        return df[df.id.str.slice(start=0, stop=len(str(id)))==str(id)]
    else:
        return df[df.id.str.slice(start=0, stop=len(str(id)))==str(id)].to_numpy()

In [17]:
#пример использования
whats_data('4016')

,id,label
21748,4016910000,коврики придверные резиновые из вулканизованно...
21749,4016910000,покрытия напольные и коврики из вулканизирован...
21750,4016910000,покрытия напольные и коврики из вулканизованно...
21751,4016910000,коврики из нетвердой непористой вулканизованно...
21752,4016910000,коврики автомобильные из непористой вулканизир...
...,...,...
260335,4016999109,части из металлонаполненных резин для обслужив...
260336,4016999109,части из металлонаполненных резин предназначен...
260337,4016999109,запасные части фронтального погрузчика sdlg из...
260338,4016999109,уплотнительный сальник из меtаллонаполненной р...
